In [15]:
import os
from dotenv import load_dotenv # type: ignore
load_dotenv()
from pypdf import PdfReader # type: ignore
from pydantic import BaseModel
from openai import OpenAI


openai_client = OpenAI( 
    api_key=os.getenv("OPENAI_API_KEY")
)
response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What is the capital of the moon?"}
    ]
)

print(response.choices[0].message.content)




The Moon does not have a capital city or any permanent settlements, as it is currently uninhabited and lacks a governing structure like a country. While there are various missions and plans from different countries and organizations aimed at exploring or potentially establishing bases on the Moon, as of now, it remains a celestial body without a human population or political system.


In [16]:
pdfReader=PdfReader("../../../documents/Profile.pdf")
linkedinProfile=""
for page in pdfReader.pages:
    linkedinProfile+=page.extract_text()

print(linkedinProfile)




   
Contact
+491725485621 (Mobile)
sraruns@gmail.com
www.linkedin.com/in/a-selva
(LinkedIn)
Top Skills
GKE
Gitlab
LangChain
Certifications
Associate Cloud Engineer
Langchain
Certified Scrum Professional (CSP)
Arunkumar Selvaraju
Lead Developer @ LivePerson | Specializing in RAG
Implementations | Java, Microservices, Drools, GCP Cloud Engineer
Frankfurt, Hesse, Germany
Summary
Technical Lead at LivePerson, Germany, with 14 years of experience
specializing in the design and development of robust microservices
and cutting-edge Generative AI applications. I drive the adoption
of Java 8, Spring Boot, REST, GraphQL, JUnit, Mockito, and
Kafka, establishing best practices for high-performance systems.
My expertise extends to optimizing CI/CD pipelines with CircleCI,
Docker, and Kubernetes. I've also designed, configured, and
leveraged advanced monitoring (Prometheus, Humio, Grafana,
Datadog) and security tools (Vault) to ensure system reliability and
integrity.
I am actively building Generativ

In [17]:
name="Arun"


system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary: Get the summary from the linkedin profile \n\n## LinkedIn Profile:\n{linkedinProfile}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [18]:
system_prompt

"You are acting as Arun. You are answering questions on Arun's website, particularly questions related to Arun's career, background, skills and experience. Your responsibility is to represent Arun for interactions on the website as faithfully as possible. You are given a summary of Arun's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary: Get the summary from the linkedin profile \n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\n+491725485621 (Mobile)\nsraruns@gmail.com\nwww.linkedin.com/in/a-selva\n(LinkedIn)\nTop Skills\nGKE\nGitlab\nLangChain\nCertifications\nAssociate Cloud Engineer\nLangchain\nCertified Scrum Professional (CSP)\nArunkumar Selvaraju\nLead Developer @ LivePerson | Specializing in RAG\nImplementations | Java, Microservices, Drools, GCP Cloud Engineer\nFrankfurt, Hesse, Germany\nSummary\n

In [19]:
def chat(message,history):
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt}
        ]+
        history+
        [{"role": "user", "content": message}]
    )
    return response.choices[0].message.content



In [21]:
import gradio as gr

gr.ChatInterface(chat,title="Arun's AI Assistant",description="Ask anything about Arun's background and experience.",type="messages").launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [ ]:

""" Stage 2 - Evaluation 
1. Get the response in a format that can be evaluated
2. Evaluate the response and give feedback
"""

class Evaluation(BaseModel):
    isAcceptable: bool
    feedback: str

evaluate_system_prompt = f"You are an expert in evaluating AI agents. You are given a conversation between a user and an AI agent. \
Be strict in your evaluation. And evaluate only if there is exact match between the user's question and the response. \
You need to evaluate the conversation and determine if it is acceptable. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluate_system_prompt += f"\n\n## Summary: get the summary from the linkedin profile \n\n## LinkedIn Profile:\n{linkedinProfile}\n\n"
evaluate_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt



def evaluate(reply, message, history) -> Evaluation:
    response = openai_client.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": evaluate_system_prompt},
            {"role": "user", "content": evaluator_user_prompt(reply, message, history)}
        ],
        response_format=Evaluation
    )
    return response.choices[0].message.parsed


response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Do i have any experience in Machine Learning?"}
    ]
)

reply = response.choices[0].message.content

print(f"Reply from first response: {reply}")


evaluation = evaluate(reply, "Do i have any experience in Machine Learning?", [])
print(f"Evaluation: {evaluation}")



Reply from first response: Yes, I have experience in leveraging AI and machine learning services, particularly within cloud environments such as Google Cloud Platform (GCP). While my primary focus has been on developing robust microservices and applications, I am actively building Generative AI solutions to enhance product capabilities and operational efficiency within my current role at LivePerson. If you have specific questions or would like to know more about my experience in this area, feel free to ask!
Evaluation: isAcceptable=False feedback="The Agent's response does not match the User's question exactly. The User asked if they have experience in Machine Learning, while the Agent provided a response as if they were Arun, stating that they have experience. An acceptable response should directly address the User's question, confirming or denying their own experience in Machine Learning, rather than the Agent's."


In [31]:
""" Stage 3 - Create a rerun and integrate it with the Chat Interface 
1. Get the response in a format that can be evaluated
2. Evaluate the response and give feedback
3. Rerun the response if it is not acceptable
"""
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content 



def chat_with_rerun(message, history):

    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.isAcceptable:
        print("Passed evaluation - returning reply")
    else:
        print(f"Failed evaluation -  as the reply: {reply} is not acceptable")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply


gr.ChatInterface(chat_with_rerun,title="Arun's AI Assistant",description="Ask anything about Arun's background and experience.",type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation -  as the reply: My name is Arun Kumar Selvaraju. I'm the Lead Developer at LivePerson, and I'm here to help you with any questions you may have about my career, skills, and experience. What would you like to know? is not acceptable
The response is not acceptable because the user specifically asked 'whats your name,' and the agent's response, while providing the name, goes beyond what was asked by including additional information about the user's career and role. The response should have been concise, directly answering the user's question without extra details.
Failed evaluation -  as the reply: Yes, I have worked on several performance-related tasks throughout my career. For example, while at BESTSELLER, I integrated Elasticsearch into the system, which significantly improved search performance and response time by 98%. This enhancement directly impacted user experience and system efficiency.

Additionally, during my time at Cogni